In [2]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [8]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break


In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [9]:
use_cuda = torch.cuda.is_available()
batch_size = 64
epochs = 14
save_model = True
log_interval = 10
dry_run = False
lr = 1.0
gamma = 0.7


torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'batch_size': batch_size}
if use_cuda:
    kwargs.update({'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True},
                 )
    
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(log_interval, dry_run, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

if save_model:
    torch.save(model.state_dict(), "mnist_cnn.pt")
    

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.293032
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.262122
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.247674
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.668838
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.355900
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.456088
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.238870
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.288156
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.612573
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.267299
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.263705
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.319757
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.164906
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.108657
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.225834
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.137867
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.389258
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.154724
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.581322
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.225412
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.147296
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.111682
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.107933
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.092833
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.167412
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.053090
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.012556
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.014172
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.020869
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.068736
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.068214
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.002585
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.011345
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.171801
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.124343
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.024783
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.013765
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.042509
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.021350
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.022041


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.035209
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.036688
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.015102
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.004214
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.019368
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.007199
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.022226
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.021058
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.009162
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.015716
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.105783
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.097637
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.002660
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.055427
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.131679
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.056553
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.014032
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.026758
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.019042
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.085938


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.025300
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.006679
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.025139
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.014803
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.038180
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.049097
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.106397
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.000683
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.002322
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.021699
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.020904
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.062176
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.009433
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.051902
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.001339
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.005222
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.097737
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.082122
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.004841
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.001670


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.020761
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.011604
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.004513
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.000696

Test set: Average loss: 0.0300, Accuracy: 9905/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.006502
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.001394
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.014195
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.165483
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.000914
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.009629
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.001960
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.001149
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.098915
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.014079
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.067835
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.199640
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.007801
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.002301
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.016808
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.009173
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.016074
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.005455
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.092795
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.003394
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.082369
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.017894
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.029686
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.051387
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.072658
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.007854
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.007197
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.014308
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.010878
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.011028
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.033200
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.001878
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.002389
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.001268
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.069345


Train Epoch: 11 [23680/60000 (39%)]	Loss: 0.026977
Train Epoch: 11 [24320/60000 (41%)]	Loss: 0.002105
Train Epoch: 11 [24960/60000 (42%)]	Loss: 0.003199
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.009075
Train Epoch: 11 [26240/60000 (44%)]	Loss: 0.073994
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.027739
Train Epoch: 11 [27520/60000 (46%)]	Loss: 0.017174
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.004322
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.006441
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.010817
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.001797
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.022994
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.030018
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.014690
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.010644
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.034853
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.002869
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.000422
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.110797
Train Epoch: 11 [35840/60000 (6

Train Epoch: 13 [5760/60000 (10%)]	Loss: 0.008572
Train Epoch: 13 [6400/60000 (11%)]	Loss: 0.013785
Train Epoch: 13 [7040/60000 (12%)]	Loss: 0.231709
Train Epoch: 13 [7680/60000 (13%)]	Loss: 0.010350
Train Epoch: 13 [8320/60000 (14%)]	Loss: 0.021174
Train Epoch: 13 [8960/60000 (15%)]	Loss: 0.100913
Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.034616
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.133686
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.004426
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.038950
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.004762
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.044644
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.001142
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.003555
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.046615
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.000289
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.050689
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.126652
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.004995
Train Epoch: 13 [17920/60000 (30%)]	Lo

Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.014574
Train Epoch: 14 [49280/60000 (82%)]	Loss: 0.001365
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.003563
Train Epoch: 14 [50560/60000 (84%)]	Loss: 0.078554
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.075271
Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.027201
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.000612
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.002738
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.050859
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.007422
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.005750
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.056962
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.010446
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.013276
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.032738
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.011799
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.000758
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.000646

Test set: Average loss: 0.0288, Accuracy: 9910/10000 (99%)



In [11]:
len(dataset1)

60000